In [1]:
# data management
import pandas as pd
from scipy.io import mmread
import scanpy as sc
import pickle
import numpy as np

# scaler
from sklearn.preprocessing import StandardScaler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# addition to models
from sklearn.calibration import CalibratedClassifierCV

# data splitting
from sklearn.model_selection import train_test_split

# scoring
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# time execution
import time

ncores = 32

In [2]:
exp_ax = mmread("../data/processed/axolotl_parts/ax_regions_data.mtx").tocsr().transpose()
meta_ax = pd.read_csv("../data/processed/axolotl_parts/ax_regions_meta.csv", index_col = 0)
gene_ax = pd.read_csv("../data/processed/axolotl_parts/ax_regions_genes.csv", index_col = 0)

Add predicted regions in whole pallium samples

In [3]:
reg_ax = pd.read_csv("../data/processed/multiome/WP_region_predictions.csv", index_col = 0)
newnames = [x.replace("-a1-1", "-1_1") for x in reg_ax.index.values]
newnames = [x.replace("-a1-2", "-1_2") for x in newnames]
newnames = [x.replace("-a3-1", "-1_3") for x in newnames]
newnames = [x.replace("-a3-2", "-1_4") for x in newnames]
reg_ax.index = newnames
meta_ax["regions_all"] = meta_ax["regions"]
meta_ax.loc[reg_ax.index.values,"regions_all"] = reg_ax["pred_regions_top"]
meta_ax.regions_all.value_counts()

medial           19483
lateral          15370
dorsal           10980
other/unknown     2303
Name: regions_all, dtype: int64

Load Div-seq data

In [4]:
exp_div = mmread("../data/processed/axolotl_parts/div_regions_data.mtx").tocsr().transpose()
meta_div = pd.read_csv("../data/processed/axolotl_parts/div_regions_meta.csv", index_col = 0)
gene_div = pd.read_csv("../data/processed/axolotl_parts/div_regions_genes.csv", index_col = 0)

Subset data to only have cells with a clear region

In [5]:
meta_reg = meta_ax.loc[meta_ax.cellclusters!="glut_SUBSET_23",:]
exp_reg = exp_ax[:,meta_ax.cellclusters!="glut_SUBSET_23"]

# make genes uniform
g_use = np.asarray(exp_reg.sum(axis = 1)>0)[:,0]
exp_reg = exp_reg[g_use,:].transpose()

# variable for startification
meta_reg["cc_reg"] = meta_reg.cellclusters+".."+meta_reg.regions
meta_reg["cc_reg"].value_counts()

<ipython-input-5-1b4c1fe41fe7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_reg["cc_reg"] = meta_reg.cellclusters+".."+meta_reg.regions


glut_SUBSET_1..whole pallium    1323
glut_SUBSET_0..whole pallium    1296
glut_SUBSET_4..medial           1020
glut_SUBSET_2..whole pallium    1008
glut_SUBSET_6..whole pallium     948
                                ... 
glut_SUBSET_4..dorsal              1
npc_SUBSET_1..lateral              1
npc_SUBSET_13..lateral             1
glut_SUBSET_11..lateral            1
npc_SUBSET_9..medial               1
Name: cc_reg, Length: 302, dtype: int64

Filter Div-seq data

In [6]:
exp_div_reg = exp_div[g_use,:].transpose()

For cc_reg with fewer than 10 occurences, will default to the region (regardless of cell type)

In [7]:
ct_regs = [x.split("..")[0] for x in meta_reg["cc_reg"].value_counts().index.values[meta_reg["cc_reg"].value_counts()<=10]]
meta_reg.loc[[x in ct_regs for x in meta_reg.cellclusters],"cc_reg"] = meta_reg.loc[[x in ct_regs for x in meta_reg.cellclusters],"regions"].values

/home/tpires/bin/miniconda3/envs/scgen-env/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Train model for all cells

Split data into train and test fractions

In [8]:
X_train, X_test, y_train, y_test = train_test_split(exp_reg, meta_reg["cellclusters"], test_size=0.2, 
                                                    random_state=42, stratify = meta_reg["cc_reg"].values)

Scale data (based on training data)

In [9]:
scaler = StandardScaler(with_mean = False)
scaler.fit(X_train)

# scale
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

# save scaler
with open("../results/Div-seq/scaler_axolotlCT_all.pkl", "wb") as f:
    pickle.dump(scaler, file=f)

Create classifiers

In [10]:
rfc = RandomForestClassifier(random_state = 1, n_estimators = 1000, n_jobs = ncores)
lr = LogisticRegression(random_state = 1, max_iter = 250, n_jobs = ncores)

Train models

In [11]:
start_time = time.time()
cal_rfc = CalibratedClassifierCV(rfc, method="sigmoid", cv=5, n_jobs = ncores)
cal_rfc.fit(X_train_sc, y_train)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_rfc_axolotlCT_all_model.pkl", "wb") as f:
    pickle.dump(cal_rfc, file=f)
    
start_time = time.time()
cal_lr = CalibratedClassifierCV(lr, method="isotonic", cv=5, n_jobs = ncores)
cal_lr.fit(X_train_sc, y_train)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_lr_axolotlCT_all_model.pkl", "wb") as f:
    pickle.dump(cal_lr, file=f)

RF 1vRest: 916.36 seconds
LR 1vRest: 8530.23 seconds


Use models to predict the test data

In [12]:
start_time = time.time()
pred_rfc = cal_rfc.predict(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

start_time = time.time()
pred_lr = cal_lr.predict(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

RF 1vRest: 23.33 seconds
LR 1vRest: 14.86 seconds


Get F1 score for predictions

In [13]:
f1_rfc = f1_score(y_test, pred_rfc, average = "macro")
print(f1_rfc)

f1_lr = f1_score(y_test, pred_lr, average = "macro")
print(f1_lr)

0.8203180404244801
0.8008689628924548


In [14]:
all_rfc = precision_recall_fscore_support(y_test, pred_rfc, zero_division = 0)
with open("../results/Div-seq/PRFSup_rfc_CT_allax.pkl", "wb") as f:
    pickle.dump(all_rfc, file=f)

all_lr = precision_recall_fscore_support(y_test, pred_lr, zero_division = 0)
with open("../results/Div-seq/PRFSup_lr_CT_allax.pkl", "wb") as f:
    pickle.dump(all_lr, file=f)

Get probabilities

In [15]:
start_time = time.time()
proba_rfc = cal_rfc.predict_proba(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc, index=y_test.index.values, columns=cal_rfc.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_rfc": pred_rfc}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_allax.csv")

start_time = time.time()
proba_lr = cal_lr.predict_proba(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr, index=y_test.index.values, columns=cal_lr.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_lr": pred_lr}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_allax.csv")

RF 1vRest: 23.56 seconds
LR 1vRest: 14.38 seconds


Get Div-seq predictions

In [16]:
# RFC
start_time = time.time()
pred_rfc_all = cal_rfc.predict(scaler.transform(exp_div_reg))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

# LR
start_time = time.time()
pred_lr_all = cal_lr.predict(scaler.transform(exp_div_reg))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

RF 1vRest: 28.33 seconds
LR 1vRest: 15.18 seconds


Get Div-seq probabilities

In [17]:
# RFC
start_time = time.time()
proba_rfc_all = cal_rfc.predict_proba(scaler.transform(exp_div_reg))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc_all, columns=cal_rfc.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_rfc": pred_rfc_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_div_all.csv")

# LR
start_time = time.time()
proba_lr_all = cal_lr.predict_proba(scaler.transform(exp_div_reg))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr_all, columns=cal_lr.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_lr": pred_lr_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_div_all.csv")

RF 1vRest: 28.11 seconds
LR 1vRest: 14.53 seconds


## Train model for neurons only

Subset the data

In [18]:
cond_neu = meta_reg["classes"]=="neuronal"
meta_neu = meta_reg.loc[cond_neu,:]
exp_neu = exp_reg[cond_neu,:]
# make genes uniform
g_use = np.asarray(exp_neu.transpose().sum(axis = 1)>0)[:,0]
exp_neu = exp_neu[:,g_use]

exp_div_neu = exp_div_reg[:,g_use]

Split data into train and test fractions

In [19]:
X_train, X_test, y_train, y_test = train_test_split(exp_neu, meta_neu["cellclusters"], test_size=0.2, 
                                                    random_state=42, stratify = meta_neu["cc_reg"].values)

Scale data (based on training data)

In [20]:
scaler = StandardScaler(with_mean = False)
scaler.fit(X_train)

# scale
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

# save scaler
with open("../results/Div-seq/scaler_axolotlCT_neu.pkl", "wb") as f:
    pickle.dump(scaler, file=f)

Create classifiers

In [21]:
rfc = RandomForestClassifier(random_state = 1, n_estimators = 1000, n_jobs = ncores)
lr = LogisticRegression(random_state = 1, max_iter = 250, n_jobs = ncores)

Train models

In [ ]:
start_time = time.time()
cal_rfc = CalibratedClassifierCV(rfc, method="sigmoid", cv=5, n_jobs = ncores)
cal_rfc.fit(X_train_sc, y_train)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_rfc_axolotlCT_neu_model.pkl", "wb") as f:
    pickle.dump(cal_rfc, file=f)
    
start_time = time.time()
cal_lr = CalibratedClassifierCV(lr, method="isotonic", cv=5, n_jobs = ncores)
cal_lr.fit(X_train_sc, y_train)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_lr_axolotlCT_neu_model.pkl", "wb") as f:
    pickle.dump(cal_lr, file=f)

RF 1vRest: 793.14 seconds


Use models to predict the test data

In [ ]:
start_time = time.time()
pred_rfc = cal_rfc.predict(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

start_time = time.time()
pred_lr = cal_lr.predict(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

Get F1 score for predictions

In [ ]:
f1_rfc = f1_score(y_test, pred_rfc, average = "macro")
print(f1_rfc)

f1_lr = f1_score(y_test, pred_lr, average = "macro")
print(f1_lr)

In [ ]:
all_rfc = precision_recall_fscore_support(y_test, pred_rfc, zero_division = 0)
with open("../results/Div-seq/PRFSup_rfc_CT_neuax.pkl", "wb") as f:
    pickle.dump(all_rfc, file=f)

all_lr = precision_recall_fscore_support(y_test, pred_lr, zero_division = 0)
with open("../results/Div-seq/PRFSup_lr_CT_neuax.pkl", "wb") as f:
    pickle.dump(all_lr, file=f)

Get probabilities

In [ ]:
start_time = time.time()
proba_rfc = cal_rfc.predict_proba(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc, index=y_test.index.values, columns=cal_rfc.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_rfc": pred_rfc}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_neuax.csv")

start_time = time.time()
proba_lr = cal_lr.predict_proba(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr, index=y_test.index.values, columns=cal_lr.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_lr": pred_lr}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_neuax.csv")

Get Div-seq predictions

In [ ]:
# RFC
start_time = time.time()
pred_rfc_all = cal_rfc.predict(scaler.transform(exp_div_neu))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

# LR
start_time = time.time()
pred_lr_all = cal_lr.predict(scaler.transform(exp_div_neu))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

Get Div-seq probabilities

In [ ]:
# RFC
start_time = time.time()
proba_rfc_all = cal_rfc.predict_proba(scaler.transform(exp_div_neu))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc_all, columns=cal_rfc.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_rfc": pred_rfc_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_div_neu.csv")

# LR
start_time = time.time()
proba_lr_all = cal_lr.predict_proba(scaler.transform(exp_div_neu))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr_all, columns=cal_lr.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_lr": pred_lr_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_div_neu.csv")

## Train model for non-NPC neurons only

Subset the data

In [ ]:
cond_dif = np.logical_or(meta_reg["subclasses"]=="Glutamatergic", meta_reg["subclasses"]=="GABAergic")
meta_dif = meta_reg.loc[cond_dif,:]
exp_dif = exp_reg[cond_dif,:]
# make genes uniform
g_use = np.asarray(exp_dif.transpose().sum(axis = 1)>0)[:,0]
exp_dif = exp_dif[:,g_use]

exp_div_dif = exp_div_reg[:,g_use]

Split data into train and test fractions

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(exp_dif, meta_dif["cellclusters"], test_size=0.2, 
                                                    random_state=42, stratify = meta_dif["cc_reg"].values)

Scale data (based on training data)

In [ ]:
scaler = StandardScaler(with_mean = False)
scaler.fit(X_train)

# scale
X_train_sc = scaler.transform(X_train)
X_test_sc = scaler.transform(X_test)

# save scaler
with open("../results/Div-seq/scaler_axolotlCT_dif.pkl", "wb") as f:
    pickle.dump(scaler, file=f)

Create classifiers

In [ ]:
rfc = RandomForestClassifier(random_state = 1, n_estimators = 1000, n_jobs = ncores)
lr = LogisticRegression(random_state = 1, max_iter = 250, n_jobs = ncores)

Train models

In [ ]:
start_time = time.time()
cal_rfc = CalibratedClassifierCV(rfc, method="sigmoid", cv=5, n_jobs = ncores)
cal_rfc.fit(X_train_sc, y_train)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_rfc_axolotlCT_dif_model.pkl", "wb") as f:
    pickle.dump(cal_rfc, file=f)
    
start_time = time.time()
cal_lr = CalibratedClassifierCV(lr, method="isotonic", cv=5, n_jobs = ncores)
cal_lr.fit(X_train_sc, y_train)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
with open("../results/Div-seq/cal_lr_axolotlCT_dif_model.pkl", "wb") as f:
    pickle.dump(cal_lr, file=f)

Use models to predict the test data

In [ ]:
start_time = time.time()
pred_rfc = cal_rfc.predict(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

start_time = time.time()
pred_lr = cal_lr.predict(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

Get F1 score for predictions

In [ ]:
f1_rfc = f1_score(y_test, pred_rfc, average = "macro")
print(f1_rfc)

f1_lr = f1_score(y_test, pred_lr, average = "macro")
print(f1_lr)

In [ ]:
all_rfc = precision_recall_fscore_support(y_test, pred_rfc, zero_division = 0)
with open("../results/Div-seq/PRFSup_rfc_CT_difax.pkl", "wb") as f:
    pickle.dump(all_rfc, file=f)

all_lr = precision_recall_fscore_support(y_test, pred_lr, zero_division = 0)
with open("../results/Div-seq/PRFSup_lr_CT_difax.pkl", "wb") as f:
    pickle.dump(all_lr, file=f)

Get probabilities

In [ ]:
start_time = time.time()
proba_rfc = cal_rfc.predict_proba(X_test_sc)
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc, index=y_test.index.values, columns=cal_rfc.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_rfc": pred_rfc}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_difax.csv")

start_time = time.time()
proba_lr = cal_lr.predict_proba(X_test_sc)
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr, index=y_test.index.values, columns=cal_lr.classes_)
pd.concat([pd.DataFrame({"y_test": y_test, "pred_lr": pred_lr}), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_difax.csv")

Get Div-seq predictions

In [ ]:
# RFC
start_time = time.time()
pred_rfc_all = cal_rfc.predict(scaler.transform(exp_div_dif))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

# LR
start_time = time.time()
pred_lr_all = cal_lr.predict(scaler.transform(exp_div_dif))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

Get Div-seq probabilities

In [ ]:
# RFC
start_time = time.time()
proba_rfc_all = cal_rfc.predict_proba(scaler.transform(exp_div_dif))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc_all, columns=cal_rfc.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_rfc": pred_rfc_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_div_dif.csv")

# LR
start_time = time.time()
proba_lr_all = cal_lr.predict_proba(scaler.transform(exp_div_dif))
print("LR 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_lr_all, columns=cal_lr.classes_, index=meta_div.index.values)
pd.concat([pd.DataFrame({"pred_lr": pred_lr_all}, index=meta_div.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_lr_CT_div_dif.csv")

## Predict 1wpi negative sample

Load trained model

In [6]:
with open("../results/Div-seq/cal_rfc_axolotlCT_all_model.pkl", "rb") as f:
    rfc_all = pickle.load(f)

Load scaler

In [7]:
with open("../results/Div-seq/scaler_axolotlCT_all.pkl", "rb") as f:
    scaler_all = pickle.load(f)

Load data

In [8]:
exp_neg = mmread("../data/processed/axolotl_parts/neg_regions_data.mtx").tocsr().transpose()
meta_neg = pd.read_csv("../data/processed/axolotl_parts/neg_regions_meta.csv", index_col = 0)
gene_neg = pd.read_csv("../data/processed/axolotl_parts/neg_regions_genes.csv", index_col = 0)

In [9]:
exp_neg_reg = exp_neg[g_use,:].transpose()

Get predictions

In [10]:
# RFC
start_time = time.time()
pred_rfc_all = rfc_all.predict(scaler_all.transform(exp_neg_reg))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))

RF 1vRest: 20.6 seconds


Get probabilities

In [11]:
# RFC
start_time = time.time()
proba_rfc_all = rfc_all.predict_proba(scaler_all.transform(exp_neg_reg))
print("RF 1vRest: %s seconds" % (round(time.time() - start_time, 2)))
probs_df = pd.DataFrame(proba_rfc_all, columns=rfc_all.classes_, index=meta_neg.index.values)
pd.concat([pd.DataFrame({"pred_rfc": pred_rfc_all}, index=meta_neg.index.values), 
           probs_df], axis = 1).to_csv("../results/Div-seq/preds_rfc_CT_neg_all.csv")

RF 1vRest: 18.98 seconds
